<a href="https://colab.research.google.com/github/AdibSadman192/Torrentuploaderanddownloader/blob/master/SharedToTBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Meta


In [0]:
!apt update && apt install -y sudo fish transmission-cli transmission-daemon wget unzip htop mediainfo ffmpeg
!transmission-daemon
!service transmission-daemon stop
!cd /root/.config/transmission-daemon/ && rm settings.json && wget "https://www.dropbox.com/s/ce29uuinixiivod/settings.json?dl=1" -O settings.json
!transmission-daemon
!mkdir -p /content/torrents
!mkdir -p /content/partials

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [88.1 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bio

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
PARTIAL_DIR = "/content/partials/"
DOWNLOAD_DIR = "/content/Downloads/"
PARTIAL_PERCENT = 25
AUTO_PROCESS_UPLOAD = True

In [0]:
CURRENT_ID = 0
PARTIALS = {}
COMPLETED = set()
COMPLETED_NON_TBD = set()
TBD = set()
CURRENT = []
PROCESSED_TO_UPLOAD = set()
CURRENTLY_PROCESSING = -1

In [0]:
def remaining_session_time():
  import time, psutil
  uptime = time.time() - psutil.boot_time()
  remain = 12*60*60 - uptime
  return remain/3600

In [0]:
def text_to_lines(text):
    import os
    result = os.linesep.join([s for s in text.splitlines() if s])
    return result.split('\n')

In [0]:
def execute_raw_cmd(cmd):
    import subprocess
    return subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8')

In [0]:
def execute_cmd(cmd):
    import subprocess
    cmd = cmd.split()
    return subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8')
    

In [0]:
def execute_cmd_sync(cmd):    
    import subprocess
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    while(True):
        # returns None while subprocess is running
        retcode = p.poll() 
        single_char = str(p.stdout.read(1).decode("utf-8"))
        yield single_char
        if retcode is not None:
            remaining = str(p.stdout.read().decode("utf-8"))
            for single_char in remaining:
              yield single_char
            break

In [0]:
def print_sync_cmd(cmd):
  import sys
  result = ""
  for single_char in execute_cmd_sync(cmd):
    if single_char == '\n':
      print(result)
      result = ""
      continue
    if len(result) >= 120:
      print("")
      result = ""
      
    result += single_char
    sys.stdout.write("\r" + result)
    sys.stdout.flush()

In [0]:
def create_torrent(to_path, torrent, verbose=True):
    import subprocess
    import os
    import sys
    
    cmd = ['transmission-create',
           '-p',
           '-t',
           "https://tracker.torrentbd.net/announce",
           '-o',
           to_path,
           torrent
          ]
    if verbose:
      print_sync_cmd(cmd)    
    else:
      execute_raw_cmd(cmd)

In [0]:
def is_tbd(id):
    tracker = execute_cmd("transmission-remote -t {} -it".format(id))
    tracker = text_to_lines(tracker)
    tracker = tracker[0].strip()
    return tracker == "Tracker 0: http://tracker.torrentbd.net:3840"

In [0]:
def is_float(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

def get_torrent_info(status, more=False):
    import re
    
    splitted = status.split()
    digits_only = re.sub("[^0-9]", "", splitted[0])
    if len(digits_only) == 0:
        id = -1
    else:
        id = int(digits_only)
        
    digits_only = re.sub("[^0-9]", "", splitted[1])
    if (len(digits_only) == 0):
        progress = 0
    else:
        progress = int(digits_only)
        
    name = splitted[-1]
    if more:
        total = len(splitted)
        up = 0.0
        down = 0.0
        ratio = 0.0
        for i in range(5, total):
            if is_float(splitted[i]) and (i+1)<total and is_float(splitted[i+1]) and (i+2) < total and is_float(splitted[i+2]):
                up = float(splitted[i])
                down = float(splitted[i+1])
                ratio = float(splitted[i+2])
                break
        return id, progress, name, up, down, ratio
    else:
        return id, progress, name

In [0]:
def update_torrent_list():
  global CURRENT
  try:
    result = execute_cmd("transmission-remote -l")
    result = text_to_lines(result)
    if (len(result) > 0):
      CURRENT = result
  except:
    pass

import time
def keep_updating_torrents():
  while True:
    try:
      update_torrent_list()
    except Exception as ex:
      print(ex)
      pass
    time.sleep(0.75)

import threading
update_thread = threading.Thread(target=keep_updating_torrents)
        
if not update_thread.isAlive():
    update_thread.start()
    print ("Torrent update thread started")
else:
    print("Torrent update thread is already running.")

def get_current_torrent_list():
  return CURRENT

Torrent update thread started


In [0]:
def get_file_info(id):
  import os

  #Returns 
  # torrent root path to create torrent
  # media file path to take snapshot
  # file lists as additional message (to use as details)
  file_info = execute_cmd("transmission-remote -t {} -if".format(id))
  file_info = text_to_lines(file_info)
  
  mx_filesize = 0.0
  mx_filepath = ""
  files = []
  for single_file in file_info[2:]:
      
      file_path = single_file[32:].strip()
      
      splitted = single_file[22:32].split()
      file_size = float(splitted[0])
      if splitted[1] == 'MB':
          file_size *= 1024.0
      elif splitted[1] == 'GB':
          file_size *= 1024.0 * 1024.0

      if mx_filesize < file_size:
          mx_filesize = file_size
          mx_filepath = file_path
      
      if single_file.endswith("RARBG.txt") or single_file.endswith("RARBG_DO_NOT_MIRROR.exe") or ("Downloaded from" in single_file):
          remove_path = os.path.join(DOWNLOAD_DIR, file_path)
          if os.path.exists(remove_path):
              os.remove(remove_path)
      else:
        files.append(single_file[22:])
  
  root_dir = mx_filepath.split('/')[0]
  filepath = os.path.basename(mx_filepath)

  message = "Title: {}\n".format(clean_title(root_dir))
  message += "File Title: {}\n\n".format(clean_title(os.path.splitext(filepath)[0]))

  message += "Directory: {}\n".format(root_dir)
  message += "Total Files: {}\n".format(len(files))
  message += "Max File Size: {} KB\nMax Size File Name: {}\n\n".format(mx_filesize, filepath)
  message += "File List:\n"
  if len(files) < 11:
      for single_file in files:
          message += single_file + "\n"
  else:
      for single_file in files[:5]:
          message += single_file + "\n"
      message += "...\n"
      for single_file in files[-5:]:
          message += single_file + "\n"

  directory = os.path.join(DOWNLOAD_DIR, root_dir)
  media_filepath = os.path.join(DOWNLOAD_DIR, mx_filepath)


  if os.path.isdir(directory):
      return directory, media_filepath, message
  else:
      return media_filepath, media_filepath, message

In [0]:
def add_torrent(torrent, dir, is_partial=True):
    import os
    global CURRENT_ID
    global TBD
    
    torrent_list = get_current_torrent_list()
    torrent_list = torrent_list[1:-1]
    total = len(torrent_list)
    
    output = execute_raw_cmd(["transmission-remote", "-a", torrent, "-w", dir])
    update_torrent_list()
    updated_list = get_current_torrent_list()
    updated_list = updated_list[1:-1]
    message = output + "\n"
    if len(updated_list) <= total:
       message += "Already exists. Please check: {}".format(torrent)
    else:
        CURRENT_ID, _, name = get_torrent_info(updated_list[-1])

        if is_tbd(CURRENT_ID):
          TBD.add(CURRENT_ID)
        
        if is_partial:
            PARTIALS[CURRENT_ID] = PARTIAL_PERCENT
            message += "Added (as temporary ID:{}): {}".format(CURRENT_ID, name) + "\n"
            cmd = "transmission-remote -t {} -d 10240".format(CURRENT_ID)
            message += "Setting max download speed to 10MB: {}".format(execute_cmd(cmd)) + "\n"
        else:
            message += "Added (as ID:{}): {}".format(CURRENT_ID, name) + "\n"
            #if not is_tbd_torrent:
            #    cmd = "transmission-remote -t {} -u 10240".format(CURRENT_ID)
            #    print("[NON TBD] Setting max upload speed to 10MB: {}".format(execute_cmd(cmd)))
    return message
    

In [0]:
def force_watch_dir(path, dir="/content/Downloads/", is_partial=False):
    import os
    from glob import glob
    torrent_files = glob(path+"/*.torrent")
    if (len(torrent_files) ==0):
        return
    for torrent_file in torrent_files:
        add_torrent(torrent_file, dir, is_partial)
        os.remove(torrent_file)

In [0]:
def show_latest(count=5):
    global TBD
    import time
    import os
    
    torrent_list = get_current_torrent_list()
    print("  " + torrent_list[0])
    torrent_list = torrent_list[1:]
    for torrent in torrent_list[-(count+1):-1]:
        if torrent == "":
            continue
        id, _, _ = get_torrent_info(torrent)
        if id in PARTIALS.keys():
            torrent = ">" + torrent[1:]
        if id in TBD:
            torrent = "*" + torrent
        else:
          torrent = " " + torrent
        print (torrent)

    if not control_thread.isAlive():
        print("Control thread was terminated. Please manually check the partial torrents.")

In [0]:
def torrent_status():
    global PROCESSED_TO_UPLOAD
    global CURRENTLY_PROCESSING

    import time
    import os
    import sys
    from IPython.display import clear_output, display

    status_show_count = 0
    while True:
        if (status_show_count%5) == 0:
            clear_output(wait=True)

            print("-"*110)
            if CURRENTLY_PROCESSING != -1:
                print("Id {} is being processed for upload.".format(CURRENTLY_PROCESSING))
            if len(PROCESSED_TO_UPLOAD) > 0:
              print("Ready To Upload Torrents: {}".format(len(PROCESSED_TO_UPLOAD)))
              for processed in PROCESSED_TO_UPLOAD:
                print(processed)
            print("-"*110)

            show_latest(50)

            print("-"*110)
            print("  ID     Done       Have  ETA           Up    Down       Max_UP                Max_DL      Total Torrents        Remaining Session Time")

        torrent_list = get_current_torrent_list()
        
        max_down = 0.0
        max_down_id = -1
        max_down_progress = 0.0
        
        max_up = 0.0
        max_up_id = -1
        max_up_ratio = 0.0
        
        for torrent in torrent_list[1:-1]:
            id, progress, name, up, down, ratio = get_torrent_info(torrent, more=True)
            if max_down < down:
                max_down = down
                max_down_id = id
                max_down_progress = progress
            if max_up < up:
                max_up = up
                max_up_id = id
                max_up_ratio = ratio
        
        result = torrent_list[-1]
        result = "  {}    {:>6} ({}:{}⬆)     {:>6} ({}:{}%)    {:>9} {:>15.2f} h       ".format(result,
                                                                                max_up,
                                                                                max_up_id, max_up_ratio,
                                                                                max_down,
                                                                                max_down_id, max_down_progress,
                                                                                len(torrent_list)-2,
                                                                                remaining_session_time())
        sys.stdout.write("\r" + result)
        sys.stdout.flush()
        status_show_count += 1
        try:
          time.sleep(1.0)    
        except:
          break

In [0]:
def image_to_bbcode(image_path):
  import requests
  import json
  url = "https://api.imgbb.com/1/upload"
  params = {'key': '6e626bc39834ef34e6d84ff2e0579736'}
  with open(image_path, 'rb') as f:
      r = requests.post(url, params=params, files={'image': f})
      if r.status_code == requests.status_codes.codes.all_ok:
        json_text = r.text
        response = json.loads(json_text)
        if 'data' in response:
          data = response['data']
          if 'image' in data and 'url' in data['image']:
            image_url = data['image']['url']

            if 'medium' in data and 'url' in data['medium']:
              medium_url = data['medium']['url']
              return "[url={image_url}][img]{medium_url}[/img][/url]".format(
                  image_url=image_url,
                  medium_url=medium_url)
            else:
              return "[url={image_url}][img]{medium_url}[/img][/url]".format(
                  image_url=image_url,
                  medium_url=image_url)

      else:
        pass
  return "Failed"

def take_snaps(to_path, file_path):
    import os
    screenshots = ""
    for i in range(1, 6):
        try:
            snap_path = os.path.join(to_path,"{}_snap{}.jpg".format(os.path.basename(file_path), i))
            cmd = ["ffmpeg",
                  "-y",
                  "-ss",
                  "00:{0:02d}:15".format(i*5),
                  "-i",
                  file_path,
                  "-vframes",
                  "1",
                  "-q:v",
                  "1",
                  snap_path
                  ]
            execute_raw_cmd(cmd)
            screenshots += image_to_bbcode(snap_path) + " "
            os.remove(snap_path)
        except Exception as e:
            print(e)

    return screenshots
        

In [0]:
def control_torrent(status):
    global COMPLETED
    global COMPLETED_NON_TBD
    import time
    id, progress, name = get_torrent_info(status)
    
    if id not in COMPLETED and progress == 100:
        if not is_tbd(id):
          COMPLETED_NON_TBD.add(id)
        COMPLETED.add(id)
    
    if id in PARTIALS.keys() and PARTIALS.get(id) <= progress:
        print ("Temporary torrent deleted at {}% (id:{}): {}".format(progress, id, name))
        cmd = "transmission-remote -t {} -rad".format(id)
        execute_cmd(cmd)
        del PARTIALS[id]
        

def control_torrents(all_status):
    for status in all_status:
        control_torrent(status)

def torrent_control():
    import time
    import os
    while True:
        try:
            torrent_list = get_current_torrent_list()
            control_torrents(torrent_list[1:-1])    
            force_watch_dir(PARTIAL_DIR, is_partial=True)
            force_watch_dir("/content/torrents/", dir=DOWNLOAD_DIR, is_partial=False)  
            time.sleep(0.5)
        except BaseException as e:
            print('{!r}; restarting thread'.format(e))

import threading
control_thread = threading.Thread(target=torrent_control)
        
if not control_thread.isAlive():
    control_thread.start()
    print ("Control thread started")
else:
    print("Control thread is already running.")

Control thread started


In [0]:
def clean_media_info(media_info):
    media_info = text_to_lines(media_info)
    
    unnecessary_keys = ["Unique ID",
                        "ID",
                        "Title",
                        "Format settings",
                        "Format/Info",
                        "Tagged date",
                        "Duration_LastFrame",
                        "Alternate group",
                        "Complete name",
                        "Cover",
                        "Attachments",
                        "Default",
                        "Forced",
                        "Encoded date",
                        "Writing application",
                        "Writing library",
                        "Color space",
                        "Chroma subsampling",
                        "Bits/(Pixel*Frame)",
                        "Language",
                        "Color range",
                        "Color primaries",
                        "Transfer characteristics",
                        "Matrix coefficients",
                        "Delay relative to video",
                        "Service kind ",
                        "Encoding settings",
                        "Statistics Tags Issue"
                       ]
    
    cleaned = ""
    for info in media_info:
        
        if info.startswith("Menu"):
            break
        
        is_necessary = True
        for unnecesarry_key in unnecessary_keys:
            if info.startswith(unnecesarry_key):
                is_necessary = False
                break
        if not is_necessary:
            continue
        
        if info.startswith("Video") or info.startswith("Audio") or info.startswith("Text"):
            cleaned += "\n"
        cleaned += info + "\n"
    
    return cleaned
    

In [0]:
def prepare_upload_torrent(torrent_path, media_filepath, details="", verbose=True):
    import os
    import sys
    
    to_path = "/content/"
    if not os.path.exists(to_path):
        os.makedirs(to_path)
    
    upload_template = '''
    
[center][df]
[size=5][font=Comic]|||||||| [/font][font=Vivaldi]Uploaded by[/font] [font=Comic]|||||||| [/font][/size]
[url=https://extraimage.net/image/yDS3][img]https://extraimage.net/images/2020/03/04/1ad511cf35a97bef9483c92d2472ffd3.th.png[/img][/url]
[/df][/center]
[hr][center][font=Curlz MT][size=7][color=#3EF60D][b]Details[/b][/color][/size][/font][/center][hr]

[b][font=consolas][color=gray][spoiler]

{details}

[/spoiler][/color][/font][/b]

[hr][center][font=Showcard Gothic][size=7][color=tomato][b]Screenshots[/b][/color][/size][/font][/center][hr]
[center]

{screenshots}

[/center]

[hr][center][size=7][color=tomato][b]𝕸𝖊𝖉𝖎𝖆 𝕴𝖓𝖋𝖔[/b][/color][/size][/center][hr]

[font=consolas][color=green]
[mediainfo]
{mediainfo}
[/mediainfo]
[/color][/font]


'''
    screenshots = take_snaps(to_path, media_filepath)   
    mediainfo = execute_raw_cmd(["mediainfo", media_filepath])
    mediainfo = clean_media_info(mediainfo)

    meta_file_path = os.path.basename(os.path.normpath(torrent_path))

    upload_text_path = os.path.join(to_path,"{}_upload.txt".format(meta_file_path))
    upload_template = upload_template.format(details=details, screenshots=screenshots, mediainfo=mediainfo)
    text_file = open(upload_text_path, "w")
    text_file.write(upload_template)
    text_file.close()     

    upload_torrent_path = os.path.join(to_path,"{}_upload.torrent".format(meta_file_path))
    create_torrent(upload_torrent_path, torrent_path, verbose=verbose)

In [0]:
def clean_title(title):
  title = title.replace(".", " ")
  title = title.replace("(", "")
  title = title.replace(")", "")
  return title

def process_to_upload(id, verbose=True):
    import os
    global COMPLETED
    global COMPLETED_NON_TBD

    if not id in COMPLETED:
        if verbose:
          print("Torrent download is not completed yet.")
        return

    if not id in COMPLETED_NON_TBD:
        if verbose:
          print("It's a torrent bd torrent.")
        return

    execute_cmd("transmission-remote -t {} -S".format(id))

    torrent_path, media_filepath, message = get_file_info(id)
    prepare_upload_torrent(torrent_path, media_filepath, message, verbose)

    remove_message = execute_cmd("transmission-remote -t {} -r".format(id))
    if verbose:
      print("Removing torrent({}): {}".format(id, remove_message))
    COMPLETED_NON_TBD.remove(id)
    

In [0]:
def remove_meta_files(directory_name):
  import os
  meta_file_path = os.path.basename(os.path.normpath(directory_name))
  txt_file_path = os.path.join("/content/", meta_file_path+"_upload.txt")
  if os.path.exists(txt_file_path):
    os.remove(txt_file_path)
  torrent_file_path = os.path.join("/content/", meta_file_path+"_upload.torrent")
  if os.path.exists(torrent_file_path):
    os.remove(torrent_file_path)

def process_torrent():
  import time
  import os

  global TBD
  global PROCESSED_TO_UPLOAD
  global CURRENTLY_PROCESSING
  
  while RUNNING_PROCESS_THREAD:
    try:
      if AUTO_PROCESS_UPLOAD:
        for ready_id in COMPLETED_NON_TBD:
          CURRENTLY_PROCESSING = ready_id
          directory_name, _, _ = get_file_info(ready_id)
          process_to_upload(ready_id, verbose=False)
          PROCESSED_TO_UPLOAD.add(directory_name)
          CURRENTLY_PROCESSING = -1
        
        if len(PROCESSED_TO_UPLOAD) > 0:
          for tbd_id in TBD:
              directory_name, _, _ = get_file_info(tbd_id)
              if directory_name in PROCESSED_TO_UPLOAD:
                remove_meta_files(directory_name)
                PROCESSED_TO_UPLOAD.remove(directory_name)
    except Exception as ex:
      print(ex)
    time.sleep(2.0)


RUNNING_PROCESS_THREAD = True
import threading
process_thread = threading.Thread(target=process_torrent)
        
if not process_thread.isAlive():
    process_thread.start()
    print ("Process thread started")
else:
    print("Process thread is already running.")

Process thread started


## Start from here


**Status Here**

In [0]:
torrent_status()

--------------------------------------------------------------------------------------------------------------
Ready To Upload Torrents: 1
/content/Downloads/Bad.Trip.2020.WEBRip.x264-ION10
--------------------------------------------------------------------------------------------------------------
  ID     Done       Have  ETA           Up    Down  Ratio  Status       Name
    1    19%   235.5 MB  3 min       90.0  3021.0    0.0  Up & Down    Sergio.2020.WEBRip.x264-ION10
--------------------------------------------------------------------------------------------------------------
  ID     Done       Have  ETA           Up    Down       Max_UP                Max_DL      Total Torrents        Remaining Session Time
  Sum:          236.7 MB             163.0  1627.0     163.0 (1:0.0⬆)     1627.0 (1:19%)            1           11.77 h       

In [0]:
!transmission-remote -t 15 -ip

**Add Here**

In [0]:
torrent = ""
print(add_torrent(torrent, dir=DOWNLOAD_DIR, is_temporary=False))

**Upload Here**


In [0]:
#process_to_upload(3, verbose=True)